### Basic Imports & Drive Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece!=0.1.90,!=0.1.91
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install colabcode

!pip install nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255317 sha256=632a4e30101f1aa5704cd0785127daeb5181784de2af2f0fe1f740052d1cac6c
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
# !pip install flask-ngrok
# !mkdir -p /root/.ngrok2/
# !echo "authtoken: 4dh9SZejMUNQpd1jvNCQ5_5bhnCu7qqp4ErVyQZ8S63" > /root/.ngrok2/ngrok.yml
!pip install pyngrok
!pip install flask_cors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
uvicorn 0.13.1 requires click==7.*, but you have click 8.1.7 which is incompatible.


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS, cross_origin
from fastapi import FastAPI
from colabcode import ColabCode


In [ ]:
from fastai import *
from fastai.basics import *
from fastai.text import *
from fastai.data import *
from fastcore.foundation import Config

from fastai.text.all import *
from fastai.callback.progress import CSVLogger

import torch
import shutil, os, pathlib
from fastapi import FastAPI
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio


### Model Details

In [ ]:
# from fastai.text.all import *

def local_initialisation():
    lang = 'te'
    backwards = False
    bs=512
    vocab_sz = 300000
    num_workers=16
    step = 'ft' # fine-tuning
    drop_mult = 1
    tsplit = 0.1

    direction = 'fwd' if backwards == False else 'bwd'

    return (lang, backwards, bs, vocab_sz, num_workers, step, drop_mult, direction, tsplit)

### Model Testing - Helper Function

In [ ]:
def standAloneTesting(TEXT, N_WORDS, N_SENTENCES, TEMPERATURE, model_path):

    print("\n\n\n------------------\nModel Testing")

    # If you want the model to work on GPU, set cpu = False - It may reduce inference time
    newLearnerObject = load_learner((model_path), cpu=True)

    output = "\n".join(newLearnerObject.predict(TEXT, int(N_WORDS), temperature=TEMPERATURE) for _ in range(int(N_SENTENCES)))

    return output

### Model Testing - Clean Output - Helper Function

In [ ]:
import re


def clean_sentence_final(sentence):
    # Remove underscores and special tokens
    cleaned_sentence = re.sub(r'_+', '', sentence)
    cleaned_sentence = cleaned_sentence.replace('▁', ' ').strip()
    cleaned_sentence = cleaned_sentence.replace('xxbos', '').strip()

    sentence = cleaned_sentence
    question_mark_position = sentence.find('?')
    fullstop_position = sentence.find('.')

    if question_mark_position != -1 and fullstop_position == -1:
      before_question_mark = sentence[:question_mark_position + 1]
      print("Before QM: ", before_question_mark)
      return before_question_mark

    elif fullstop_position != -1 and question_mark_position == -1:
      before_full_stop = sentence[:fullstop_position + 1]
      print("Before FS: ", before_full_stop)
      return before_full_stop

    elif fullstop_position != -1 and question_mark_position != -1:
      if (fullstop_position < question_mark_position):
        before_full_stop = sentence[:fullstop_position + 1]
        print("Before FS: ", before_full_stop)
        return before_full_stop
      else:
        before_question_mark = sentence[:question_mark_position + 1]
        print("Before QM: ", before_question_mark)
        return before_question_mark

    else:
      return sentence



def cleanTesting(TEXT, N_WORDS, N_SENTENCES, TEMPERATURE, version, model_path):

  print("\n\n\n------------------\nModel Testing")

  lang, backwards, bs, vocab_sz, num_workers, step, drop_mult, direction, tsplit = local_initialisation()
  # newLearnerObject = load_learner((model_path/f'ft_v{version}_tsplit{tsplit}_lr{lr}_epoch{epochs}.pkl'), cpu=False)
  newLearnerObject = load_learner((model_path), cpu=True)

  output = []

  for i in range(N_SENTENCES):
    print("Prossesing Sentence" , (i+1))
    predictionOutput = newLearnerObject.predict(TEXT, int(N_WORDS), temperature=TEMPERATURE)
    print(clean_sentence_final(predictionOutput))

### Testing Model

In [ ]:
N_WORDS = 2
N_SENTENCES = 5
TEMPERATURE = 1 # From 0 (lower) to 1 - https://lukesalamone.github.io/posts/what-is-temperature/
# Feel free to to experiment with values above


# The model_path is sent for loading the model - You may need to change it as per your gDrive structure
model_path = pathlib.Path('/content/drive/MyDrive/ft_v9_tsplit0.1_lr0.1_epoch1.pkl')
newLearnerObject = load_learner((model_path), cpu=True)
# model_path = pathlib.Path('./ft_v9_tsplit0.1_lr0.1_epoch1.pkl')

In [ ]:
TEXT = "పుట్టిన రోజు "
output = standAloneTesting(TEXT, N_WORDS, N_SENTENCES, TEMPERATURE, model_path)

output = output.split('\n')

for response in output:
  print(response)




------------------
Model Testing


xxunk ▁పుట్టిన ▁రోజు ▁శుభాకాంక్షలు .
xxunk ▁పుట్టిన ▁రోజు ▁ జరుపుకుంటున్నారు
xxunk ▁పుట్టిన ▁రోజు . ▁మరియు
xxunk ▁పుట్టిన ▁రోజు నాడు ▁ఈ
xxunk ▁పుట్టిన ▁రోజు ▁పండగ .


In [ ]:

app=FastAPI()

@app.get("/")
async def read():
  return {"Hello": "words"}

In [ ]:
port_no = 5000

In [ ]:
# app = FastAPI()
# @app.post('/teluguprediction')
# def telugu_prediction():

#     input_data = input_parameters.json()
#     input_dictionary = json.loads(input_data)

#     text=input_dictionary['text']

#     output = standAloneTesting(TEXT, N_WORDS, N_SENTENCES, TEMPERATURE, model_path)
#     return output
app = Flask(__name__)
# CORS(app, resources={r"/api/*": {"origins": "http://127.0.0.1:5500/"}})
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'


ngrok.set_auth_token("2dPPWCFZBOTUl7SW7eqE17MOWCT_4JBWSnv2Y3Gem26uCZrZD")
public_url =  ngrok.connect(port_no).public_url

@app.route(rule="/", methods=["GET", "POST"])
def handle_request():
    # The GET endpoint
    if request.method == "GET":
        return "This is the GET"

    # The POST endpoint
    if request.method == "POST":
        # accesing the pass ed payload
        input_data = request.get_json()
        # input_dictionary = json.loads(input_data)

        text=input_data['text']
        print(text)
        # output = standAloneTesting(text, N_WORDS, N_SENTENCES, TEMPERATURE, model_path)
        output = "\n".join(newLearnerObject.predict(text, int(N_WORDS), temperature=TEMPERATURE) for _ in range(int(N_SENTENCES)))
        return str(output)

print(f"To access the Gloable link please click {public_url}")

app.run(port=port_no)

To access the Gloable link please click https://35f7-35-227-98-168.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


పుట్టిన రోజు 


INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 14:19:41] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 14:19:53] "POST / HTTP/1.1" 415 -


పుట్టిన రోజు 


...

INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 14:27:20] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 14:30:14] "OPTIONS / HTTP/1.1" 200 -


పుట్టిన రోజు 


INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 14:30:16] "POST / HTTP/1.1" 200 -


పుట్టిన రోజు 


INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 14:37:42] "POST / HTTP/1.1" 200 -


In [ ]:
# ngrok_tunnel = ngrok.connect('8000')
# print('Public URL:', ngrok_tunnel.public_url)
# nest_asyncio.apply()
# uvicorn.run(app, port=8000)
# listener = ngrok.forward(8000, authtoken='usr_13CBp4HFdH6RYRukz3vi30VB5bq')
# print(f"Ingress established at {listener.url()}")


In [ ]:
# Click on RAM utilization on right hand side

In [ ]:
lang, backwards, bs, vocab_sz, num_workers, step, drop_mult, direction, tsplit = local_initialisation()
newLearnerObject = load_learner(model_path, cpu=True)

# Creates a LearnerObejct in the memory - Check for the RAM utilization to visualise how much spike occured

### Number of parameters

In [ ]:
# Calculate the number of parameters in the model
total_params = sum(p.numel() for p in newLearnerObject.model.parameters())
print(f"Total number of parameters: {total_params}")

### Memory Usage

In [ ]:
!pip install memory_profiler
%load_ext memory_profiler

# Profile memory before executing the function
%memit -r 1 -c newLearnerObject.predict(TEXT, int(N_WORDS), temperature=TEMPERATURE)

### Inference Time

In [ ]:
TEXT = "ఆ తర్వాత పరిస్థితులు"
%timeit newLearnerObject.predict(TEXT, int(N_WORDS), temperature=TEMPERATURE)